In [2]:
import pandas as pd
import time
import numpy as np
from enum import IntEnum
from datetime import datetime as dt
import os
import seaborn as sns
import xlsxwriter
from matplotlib import pyplot as plt

In [3]:
def match_scaffold(k, specie, scaffold, min):
    df_k_mers = pd.read_csv('../Data/Intermediate/' + str(k) + '_mers.csv')

    sp_sca = specie + '_' + scaffold

    df_query = df_k_mers[df_k_mers['Specie_Scaffold'].str.contains(sp_sca, case=False)]

    df_query['Specie_Scaffold'] = df_query['Specie_Scaffold'].str.replace("\['", "")
    df_query['Specie_Scaffold'] = df_query['Specie_Scaffold'].str.replace("'\]", "")
    
    sep = df_query['Specie_Scaffold'].str.split("', '", expand=True)
    df_query = sep.merge(df_query[str(k) + '_mers'], left_index=True, right_index=True, how='right')
    melt = pd.melt(df_query, id_vars=[str(k) + '_mers']).dropna().drop('variable',1).rename(columns = {'value':'Specie_Scaffold'})
    
    match = melt.groupby('Specie_Scaffold', as_index=False).count().rename(columns={str(k) + '_mers':'# Appearances'}).sort_values('# Appearances', ascending=False)
    match[['Specie', 'Scaffold']] = match['Specie_Scaffold'].str.split("_", n = 1, expand=True)
    match = match.loc[match['# Appearances'] > min, ['Specie', 'Scaffold', '# Appearances']]

    return match  
    

In [5]:
acc = match_scaffold( 8, 'human', 'NC_000005.10', 5)
acc

C:\Users\bertr\AppData\Local\Temp\ipykernel_12928\1810942397.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df_query['Specie_Scaffold'] = df_query['Specie_Scaffold'].str.replace("\['", "")
C:\Users\bertr\AppData\Local\Temp\ipykernel_12928\1810942397.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query['Specie_Scaffold'] = df_query['Specie_Scaffold'].str.replace("\['", "")
C:\Users\bertr\AppData\Local\Temp\ipykernel_12928\1810942397.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df_query['Specie_Scaffold'] = df_query['Specie_Scaffold'].str.replace("'\]", "")
C:\Users\bertr\AppData\Local\Temp\ipykernel_12928\1810942397.py:9: Setting

,Specie,Scaffold,# Appearances
53,Human,NC_000005.10,767
79,Papio,NC_018157.1,531
52,Gorilla,NC_018429.2,310
65,Monodelphis,NC_008803.1,93
64,Monodelphis,NC_008801.1,77
38,Croco,NW_017728909.1,59
67,Orca,NW_004438453.1,57
57,Koala,NW_018343955.1,41
0,Aadvark,NW_006921643.1,37
39,Croco,NW_017728914.1,31


In [6]:
acc.groupby('Specie')['Scaffold'].first()

Specie
Aadvark        NW_006921643.1
Alligator M    NW_017712138.1
Chelonia       NW_006646735.1
Chrysemys      NW_007281339.1
Croco          NW_017728909.1
Devil          NW_003816605.1
Gorilla           NC_018429.2
Human            NC_000005.10
Koala          NW_018343955.1
Monodelphis       NC_008803.1
Orca           NW_004438453.1
Ostrich        NW_009271620.1
Papio             NC_018157.1
Pelodiscus     NW_005871017.1
Pogona         NW_018150745.1
Name: Scaffold, dtype: object

In [5]:
# To load all of them at once and afterwards accessing one by one, this might be an option.
#https://towardsdatascience.com/a-simple-trick-to-load-multiple-excel-worksheets-in-pandas-3fae4124345b
# Define filepath
filepath = '../Data/Raw/Tables_Filtered_IK_format.xlsx'

# Load Excel file using Pandas with `sheet_name=None`
df_dict = pd.read_excel(filepath, sheet_name=None)

# Preview
#df_dict

# Get a specific one
#human = df_dict.get('Human')

# aprox 3 min 40 secs

In [6]:
# -*- coding: utf-8 -*-

"""

LIFE97011 - Computing
Python Programming - Assessed Exercise No. 3
Task: Smith Waterman local alignment
@Author: Slaviana Pavlovich

"""

# Assigning the constants for the scores
class Score(IntEnum):
    # order by value it should be
    MATCH = 10      
    CONS_GAP = -2   # Want to penalize worst a first gap than a secong gap
    FIRST_GAP = -4
    MISMATCH = -3
    #INV = 0
    DUP = 1

# Match: 10 , First_gap: -4, cons_gap: -1, mismatch: -5

# Assigning the constant values for the traceback
class Trace(IntEnum):
    STOP = 0
    LEFT = 1 
    UP = 2
    DIAGONAL = 3

def excel_reader(specie):
    data = pd.read_excel('../Data/Raw/Tables_Filtered_IK.xlsx', specie)
    
    data['Gene'] = data['Locus'].str.split('(\d+)').str[0] + data['Strand']
    data['Gene_non_or'] = data['Locus'].str.split('(\d+)').str[0]
    data.reset_index(inplace= True)

    data = data[data['Gene'].str.contains('LOC') == False]  

    return data

# Implementing the Smith Waterman local alignment
def smith_waterman(seq_1, seq_2):

    # Data structure:
    # 0: index
    # 1: gen + orientation
    # 2: gen

    seq_1 = seq_1[['index', 'Gene', 'Gene_non_or']]
    seq_2 = seq_2[['index', 'Gene', 'Gene_non_or']]

    row = len(seq_1) + 1
    col = len(seq_2) + 1
    matrix = np.zeros(shape=(row, col), dtype= int)  
    tracing_matrix = np.zeros(shape=(row, col), dtype= int)  

    # Initialising the variables to find the highest scoring cell
    max_score = -1
    max_index = (-1, -1)

    diagonal_score = 0
    vertical_score = 0
    horizontal_score = 0
    
    # Calculating the scores for all cells in the matrix

    for i in range(1, row):
        for j in range(1, col):
            
            # Calculating the diagonal score (match score)
            # If there is a match, always win? And then, we can skip all further comparations?
            match_value = Score.MATCH if seq_1.iloc[i - 1, 2] == seq_2.iloc[j - 1, 2] else Score.MISMATCH
            #Score.INV if seq_1.iloc[i - 1, 2] == seq_2.iloc[j - 1, 2] else  --> No sense, it will make sense if the overall is inversed and then one of them is turned around. So, first, check the overall waz and then compare them.
                
            diagonal_score = matrix[i - 1, j - 1] + match_value

            # Calculating the vertical gap score, penalizing less consecutive gaps and identifying duplicates
            #if seq_2.iloc[j-1, 2] == seq_2.iloc[j - 2, 2]:
                #vertical_score = matrix[i - 1, j] + Score.DUP elif
            if matrix[i - 1, j] == vertical_score:
                vertical_score = matrix[i - 1, j] + Score.CONS_GAP
            else: vertical_score = matrix[i - 1, j] + Score.FIRST_GAP
            
            # Calculating the vertical gap score, penalizing consecutive gaps and identifying duplicates
            #if seq_1.iloc[i-1, 2] == seq_1.iloc[i-2, 2]:
                #horizontal_score = matrix[i, j - 1] + Score.DUP elif
            if matrix[i, j - 1] == horizontal_score:
                horizontal_score = matrix[i, j - 1] + Score.CONS_GAP
            else: horizontal_score = matrix[i, j - 1] + Score.FIRST_GAP
            
            # Taking the highest score 
            matrix[i, j] = max(0, diagonal_score, vertical_score, horizontal_score)
            
            # Tracking where the cell's value is coming from    
            if matrix[i, j] == 0: 
                tracing_matrix[i, j] = Trace.STOP
                
            elif matrix[i, j] == horizontal_score: 
                tracing_matrix[i, j] = Trace.LEFT
                
            elif matrix[i, j] == vertical_score: 
                tracing_matrix[i, j] = Trace.UP
                
            elif matrix[i, j] == diagonal_score: 
                tracing_matrix[i, j] = Trace.DIAGONAL 
                
            # Tracking the cell with the maximum score
            # If we want different strings, here we can define a threshold and keep track of all the higher values
            if matrix[i, j] >= max_score:
                max_index = (i,j)
                max_score = matrix[i, j]


    # Initialising the variables for tracing
    aligned_seq_1 = [ ]
    aligned_seq_2 = [ ]
    index_seq_1 = [ ]
    index_seq_2 = [ ]
    (max_i, max_j) = max_index

    # Tracing and computing the pathway with the local alignment
    #if max_score > 3 * Score.MATCH
    while tracing_matrix[max_i, max_j] != Trace.STOP:
        if tracing_matrix[max_i, max_j] == Trace.DIAGONAL:
            aligned_seq_1.insert(0, seq_1.iloc[max_i - 1, 2])
            aligned_seq_2.insert(0, seq_2.iloc[max_j - 1, 2])

            index_seq_1.insert(0, seq_1.iloc[max_i - 1, 0])
            index_seq_2.insert(0, seq_2.iloc[max_j - 1, 0])

            max_i = max_i - 1
            max_j = max_j - 1
            

        elif tracing_matrix[max_i, max_j] == Trace.UP:
            aligned_seq_1.insert(0, seq_1.iloc[max_i - 1, 2])
            aligned_seq_2.insert(0, '-')

            index_seq_1.insert(0, seq_1.iloc[max_i - 1, 0])
            index_seq_2.insert(0, '-')

            max_i = max_i - 1    
            

        elif tracing_matrix[max_i, max_j] == Trace.LEFT:
            aligned_seq_1.insert(0, '-')
            aligned_seq_2.insert(0, seq_2.iloc[max_j - 1, 2])

            index_seq_1.insert(0, '-')
            index_seq_2.insert(0, seq_2.iloc[max_j - 1, 0])
            
            max_j = max_j - 1

    return aligned_seq_1, aligned_seq_2, index_seq_1, index_seq_2, max_score, max_index, max_i, max_j, matrix

def duplicate(all):
    for i in range(1, len(all)):
        if all.loc[i, 'Result'] == 'Gap':
            if (all.loc[i-1, 'Result'] == 'Match') | (all.loc[i-1, 'Result'] == 'Inversion'):
                if (all.loc[i-1,'Query Sequence Gene'] == all.loc[i,'Query Sequence Gene']) | (all.loc[i-1,'Target Sequence Gene'] == all.loc[i,'Target Sequence Gene']):
                    all.loc[i, 'Result'] = 'Duplicate'

                    # Check for consecutives duplicates
                    j = i+1
                    while((j<len(all)) & (all.loc[j, 'Result'] == 'Gap') & (all.loc[i,'Query Sequence Gene'] == all.loc[j,'Query Sequence Gene']) & (all.loc[i,'Target Sequence Gene'] == all.loc[j,'Target Sequence Gene'])):
                        all.loc[j, 'Result'] = 'Duplicate'
                        j+=1
                    i=j-1
    return all

def merge(seq_1, seq_2, aligned_seq_1, index_seq_1, aligned_seq_2, index_seq_2):
    align_seq_1 = pd.DataFrame({'Original Position' : index_seq_1, 'Gene': aligned_seq_1}).merge(seq_1[['index', '#Replicon Name', 'Replicon Accession', 'Strand']], left_on = 'Original Position', right_on = 'index', how = 'left')
    align_seq_2 = pd.DataFrame({'Original Position' : index_seq_2, 'Gene': aligned_seq_2}).merge(seq_2[['index', '#Replicon Name', 'Replicon Accession', 'Strand']], left_on = 'Original Position', right_on = 'index', how = 'left')

    align_seq_1.rename(columns={'Original Position': 'Query Sequence Original Position', 'Gene' : 'Query Sequence Gene', '#Replicon Name': 'Query Sequence Replicon Name', 'Replicon Accession' : 'Query Sequence Replicon Accession', 'Strand': 'Query Sequence Orientation'}, inplace = True)
    align_seq_2.rename(columns={'Original Position': 'Target Sequence Original Position', 'Gene' : 'Target Sequence Gene', '#Replicon Name': 'Target Sequence Replicon Name', 'Replicon Accession' : 'Target Sequence Replicon Accession', 'Strand': 'Target Sequence Orientation'}, inplace = True)

    all = align_seq_1.loc[:, align_seq_1.columns != 'index'].merge(align_seq_2.loc[:, align_seq_2.columns != 'index'], right_index = True, left_index = True)

    all['Result'] = np.where(all['Query Sequence Gene'] == all['Target Sequence Gene'],  
                    np.where(all['Query Sequence Orientation'] == all['Target Sequence Orientation'], 'Match', 'Inversion'),
                    np.where(((all['Query Sequence Gene'] == '-') | (all['Target Sequence Gene'] == '-')),
                      'Gap', 'Mismatch'))
    
    all = all.reindex(
            columns = ['Query Sequence Original Position', 'Query Sequence Replicon Name',
            'Query Sequence Replicon Accession', 'Query Sequence Orientation', 'Query Sequence Gene', 'Result', 'Target Sequence Gene', 'Target Sequence Orientation',
            'Target Sequence Replicon Accession', 'Target Sequence Replicon Name', 'Target Sequence Original Position'])

    return all


In [7]:
# Data cleaning for each specie
df_species = pd.DataFrame()

species = df_dict.keys()

for s in species:
    aux = df_dict.get(s)
    aux['Specie'] = s
    df_species = pd.concat([df_species, aux])

df_species['Gene'] = df_species['Locus'].str.split('(\d+)').str[0] + df_species['Strand']
df_species['Gene_non_or'] = df_species['Locus'].str.split('(\d+)').str[0]
df_species.reset_index(inplace= True)

df_species = df_species[df_species['Gene'].str.contains('LOC') == False]  

In [ ]:
acc = match_scaffold( 8, 'human', 'NC_000005.10', 5)

In [8]:
acc = acc.reset_index()

In [9]:
target_specie = acc.iloc[0,1]
target_scaffold = acc.iloc[0,2]

target_seq = df_species[(df_species['Specie'] == target_specie) & (df_species['Replicon Accession'] == target_scaffold)]
target_seq_name = target_specie + '_' + target_scaffold
target_seq_or = target_seq

# Save our results afterwards
path = '../Data/S_W_Intermediate/Scaffold/' # + query_specie + '_' + target_specie + '_' + dt.now().strftime('%Y%m%d_%H%M%S') + '.xlsx'
timestamp = dt.now().strftime('%Y%m%d_%H%M%S')
writer = pd.ExcelWriter(path + target_specie + '_' + target_scaffold + '_' + timestamp + '.xlsx', engine = 'xlsxwriter')

#os.makedirs(path)

# Parameters
params = []
params = pd.DataFrame({
    'Description': ['', 'Parameters', 'Match', 'Mismatch', 'First gap', 'Second gap'], 
    'Value': ['', '', Score.MATCH, Score.MISMATCH, Score.FIRST_GAP, Score.CONS_GAP]})

results = []
results = pd.DataFrame(data = {'Specie':'', 'Scaffold':'', '% Covered Target sequence':'', '% Covered Scaffold': '', 'Number of consecutive genes': ''})
#results = pd.DataFrame(data = {'Accession', '% Covered Target sequence', '% Covered Accession', 'Number of consecutive genes'})

acc = acc.drop(0).sort_values(['Specie', '# Appearances'], ascending=False)

for i in range(0, len(acc)):
    
    if acc.iloc[i, 1] != acc.iloc[i-1, 1]:
        target_seq = target_seq_or
        
    query_specie = acc.iloc[i, 1]
    query_scaffold = acc.iloc[i,2]

    query_seq = df_species.loc[(df_species['Specie'] == query_specie) & (df_species['Replicon Accession'] == query_scaffold)]
    query_seq_name = query_specie + '_' + query_scaffold

    print(query_seq_name)

    # Executing the Smith Waterman local alignment algorithm
    aligned_query_seq = []
    aligned_target_seq = []
    index_query_seq = []
    index_target_seq = []
    matrix = [[]]

    tic = time.perf_counter()
    aligned_query_seq, aligned_target_seq, index_query_seq, index_target_seq, max_score, max_index, max_i, max_j, matrix = smith_waterman(query_seq, target_seq)
    toc = time.perf_counter()

    # Time
    d = toc - tic
    print('Computed in %s'%time.strftime('%H:%M:%S', time.gmtime(d)))

    all = []

    all = merge(query_seq, target_seq, aligned_query_seq, index_query_seq, aligned_target_seq, index_target_seq)

    all = duplicate(all)

    index_query_seq_no_gaps = []
    index_target_seq_no_gaps = []
    index_query_seq_no_gaps = list(filter(lambda c: c!= '-', index_query_seq))
    index_target_seq_no_gaps = list(filter(lambda c: c!= '-', index_target_seq))

    summary = []
    summary = pd.DataFrame({
        'Description' : ['Query Sequence', 'Length Query Sequence', 'Target Sequence', 'Length Target Sequence', 'Score', '', 'Aligned Query Sequence initial position', 'Aligned Query Sequence final position', 'Aligned Query Sequence length', '', 'Aligned Target Sequence initial position', 'Aligned Target Sequence final position', 'Aligned Target Sequence length'],
        'Value': [query_seq_name, len(query_seq), target_seq_name, len(target_seq), max_score, '', index_query_seq_no_gaps[0], 
                    index_query_seq_no_gaps[-1], index_query_seq_no_gaps[-1] - index_query_seq_no_gaps[0], '', index_target_seq_no_gaps[0], 
                    index_target_seq_no_gaps[-1], index_target_seq_no_gaps[-1] - index_target_seq_no_gaps[0]]
    })

    summary = pd.concat([summary, params])

    aligned_query_seq_no_gaps = []
    aligned_target_seq_no_gaps = []
    aligned_query_seq_no_gaps = list(filter(lambda c: c!= '-', aligned_query_seq))
    aligned_target_seq_no_gaps = list(filter(lambda c: c!= '-', aligned_target_seq)) 

    #KPIs
    perc_align_query_seq = len(aligned_query_seq_no_gaps) / len(query_seq)
    perc_align_target_seq = len(aligned_target_seq_no_gaps) / len(target_seq)

    kpi = []
    kpi = pd.DataFrame({
        'Description': ['','KPIs','% Covered Target sequence', '% Covered Scaffold', 'Number of consecutive genes', ''], #include where duplicates occur
        'Value': ['','', perc_align_target_seq, perc_align_query_seq, len(all), '']
    })

    result_perc = all.groupby('Result').count() / all.shape[0]
    result_perc = result_perc.reset_index().iloc[:,[0,1]]
    result_perc.rename(columns = {'Result':'Description', 'Query Sequence Original Position': 'Value'}, inplace=True)

    summary = pd.concat([summary, kpi, result_perc]) # summary.append(kpi)

    # Write
    summary.to_excel(writer, sheet_name = 'Summary_' + query_scaffold, index = False, header=False)
    all.to_excel(writer, sheet_name = 'Comparison_' + query_scaffold, index = False)

    actual_results = pd.DataFrame(data = {'Specie':[query_specie], 'Scaffold': [query_scaffold], '% Covered Target sequence': [perc_align_target_seq], '% Covered Accession': [perc_align_query_seq], 'Number of consecutive genes': [len(all)]})
                                
    results = pd.concat([results, actual_results])
    
    heatmap = sns.heatmap(matrix)
    plt.title('Heatmap of ' + target_seq_name + ' vs. ' + query_seq_name, fontsize = 12)
    plt.savefig(path + 'Images/' + timestamp + '_' + query_seq_name + '.png')
    plt.clf()

    workbook  = writer.book
    worksheet = writer.sheets['Summary_' + query_seq_name]
    worksheet.insert_image('H2', path + 'Images/' + timestamp + '_' + query_seq_name + '.png')

    # Remove match parts, when we have the same specie; and clean variables
    prev = []
    after = []
    prev = target_seq.loc[target_seq['index'] < index_target_seq_no_gaps[0] + 1]
    after = target_seq.loc[index_target_seq_no_gaps[-1] - 1 < target_seq['index']]

    target_seq = pd.concat([prev, after])
    del heatmap, perc_align_target_seq, perc_align_query_seq, result_perc

results.to_excel(writer, sheet_name = 'Results', index = False)
writer.sheets['Results'].activate()
writer.save()   


Pogona_NW_018150745.1
Computed in 00:00:03
Pogona_NW_018150707.1
Computed in 00:00:03
Pogona_NW_018150815.1
Computed in 00:00:03
Pelodiscus_NW_005871017.1
Computed in 00:00:05
Pelodiscus_NW_005871047.1
Computed in 00:00:04
Papio_NC_018157.1
Computed in 00:01:01
Ostrich_NW_009271620.1
Computed in 00:00:04
Orca_NW_004438453.1
Computed in 00:00:06
Orca_NW_004438462.1
Computed in 00:00:03
Orca_NW_004438529.1
Computed in 00:00:02
Orca_NW_004438436.1
Computed in 00:00:03
Orca_NW_004438483.1
Computed in 00:00:02
Monodelphis_NC_008803.1
Computed in 00:02:18
Monodelphis_NC_008801.1
Computed in 00:03:50
Koala_NW_018343955.1
Computed in 00:00:10
Koala_NW_018343957.1
Computed in 00:00:05
Koala_NW_018344103.1
Computed in 00:00:02
Gorilla_NC_018429.2
Computed in 00:01:36
Devil_NW_003816605.1
Computed in 00:00:02
Devil_NW_003816604.1
Computed in 00:00:01
Devil_NW_003816613.1
Computed in 00:00:02
Croco_NW_017728909.1
Computed in 00:00:24
Croco_NW_017728914.1
Computed in 00:00:12
Croco_NW_017728886.1
C

<Figure size 640x480 with 0 Axes>

In [14]:
writer.save()

c:\Users\bertr\anaconda3\envs\thesis\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
